<a href="https://colab.research.google.com/github/Shujaat123/ADMRILSE/blob/main/Alzheimer_Disease_Classification_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **AD-MRI-LSE: Alzheimer Disease Classification From Structural MR Images Using Deep Latent Space Encoding**


This code provide Tensorflor-Keras implementation of AD-MRI-LSE algorithm.

# Loading Useful packages

In [ ]:
## Load useful packages
!pip install wget
from random import sample

import keras
# import os.path
from os import path, listdir
import h5py
import keras.backend as K
import numpy as np
import tensorflow as tf
import wget
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.layers import *
from keras.models import Model
from keras.models import load_model
from keras.utils.np_utils import to_categorical
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from scipy.io import loadmat

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=bdf2e4602a82b5ea2f30c306d785b9ccb8247d45a141e2d04b3b0e5b91e45137
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [ ]:
!python --version
print('Tensorflow:',tf.__version__)
print('Keras:',keras.__version__)


Python 3.7.10
Tensorflow: 2.4.1
Keras: 2.4.3


# Downloading dataset

**Loading and processing dataset**

In [ ]:
classes = ('AD_F','AD_M','NC_F','NC_M')
# num_AD_F = 137
# num_AD_M = 157
# num_NC_F = 186
# num_NC_M = 187

num_AD_F = 10 #137
num_AD_M = num_AD_F
num_NC_F = num_AD_F
num_NC_M = num_AD_F

num_samples = num_AD_F+num_AD_M+num_NC_F+num_NC_M
num_train = np.round(0.8*num_samples/4)
num_val = np.round(0.1*num_samples/4)


AD_F_labels = np.zeros((num_AD_F,1))
AD_M_labels = np.zeros((num_AD_M,1))
NC_F_labels = np.ones((num_NC_F,1))
NC_M_labels = np.ones((num_NC_M,1))

Class_labels = np.concatenate((AD_F_labels,AD_M_labels,NC_F_labels,NC_M_labels), axis = 0)

data_path = 'https://raw.githubusercontent.com/Shujaat123/ADMRILSE/master/ADNI_mni_normalized_rescaled_DB/'

AD_F_scans = []
AD_M_scans = []
NC_F_scans = []
NC_M_scans = []

for Cname in classes:
  if(Cname=='AD_F'):
    num_sub = num_AD_F
    flist = []
  elif(Cname=='AD_M'):
    num_sub = num_AD_M
    flist = []
  elif(Cname=='NC_F'):
    num_sub = num_NC_F
    flist = []
  else:
    num_sub = num_NC_M
    flist = []
  
  for sub_loop in range(0,num_sub):
    # print("{:0>3d}".format(sub_loop+1))
    filename = Cname+'_subject_'+'{:0>3d}'.format(sub_loop+1)+'.mat'
    # print(filename)
    # if(path.exists(filename)):
    #   !rm $filename
    #   print('existing file:', filename, ' has been deleted')
    # print('downloading latest version of file:', filename)
    # file_path = data_path + Cname + '/' + filename
    # wget.download(file_path, filename)
    if(not(path.exists(filename))):
      print('downloading latest version of file:', filename)
      file_path = data_path + Cname + '/' + filename
      wget.download(file_path, filename)

    print('Loading data file:', filename)  
    MR_img = loadmat(filename)['re_mri_scan']
    flist.append(MR_img)
  
  if(Cname=='AD_F'):
    AD_F_scans=np.asarray(flist)
  elif(Cname=='AD_M'):
    AD_M_scans=np.asarray(flist)
  elif(Cname=='NC_F'):
    NC_F_scans=np.asarray(flist)
  else:
    NC_M_scans=np.asarray(flist)
print('DONE')
!ls


downloading latest version of file: AD_F_subject_001.mat
Loading data file: AD_F_subject_001.mat
downloading latest version of file: AD_F_subject_002.mat
Loading data file: AD_F_subject_002.mat
downloading latest version of file: AD_F_subject_003.mat
Loading data file: AD_F_subject_003.mat
downloading latest version of file: AD_F_subject_004.mat
Loading data file: AD_F_subject_004.mat
downloading latest version of file: AD_F_subject_005.mat
Loading data file: AD_F_subject_005.mat
downloading latest version of file: AD_F_subject_006.mat
Loading data file: AD_F_subject_006.mat
downloading latest version of file: AD_F_subject_007.mat
Loading data file: AD_F_subject_007.mat
downloading latest version of file: AD_F_subject_008.mat
Loading data file: AD_F_subject_008.mat
downloading latest version of file: AD_F_subject_009.mat
Loading data file: AD_F_subject_009.mat
downloading latest version of file: AD_F_subject_010.mat
Loading data file: AD_F_subject_010.mat
downloading latest version of 

In [ ]:
AD_F_list = list(range(0,num_AD_F))
AD_M_list = list(range(0,num_AD_M))
NC_F_list = list(range(0,num_NC_F))
NC_M_list = list(range(0,num_NC_M))
total_list = AD_F_list + AD_M_list + NC_F_list + NC_M_list

print('Number of \'AD_F\' samples:',len(AD_F_list))
print('Number of \'AD_M\' samples:',len(AD_M_list))
print('Number of \'NC_F\' samples:',len(NC_F_list))
print('Number of \'NC_M\' samples:',len(NC_M_list))
print('Total number of samples:',len(total_list))

print(AD_F_scans.shape)
print(AD_M_scans.shape)
print(NC_F_scans.shape)
print(NC_M_scans.shape)

print(AD_F_labels.shape)
print(AD_M_labels.shape)
print(NC_F_labels.shape)
print(NC_M_labels.shape)


Number of 'AD_F' samples: 10
Number of 'AD_M' samples: 10
Number of 'NC_F' samples: 10
Number of 'NC_M' samples: 10
Total number of samples: 40
(10, 120, 160, 120)
(10, 120, 160, 120)
(10, 120, 160, 120)
(10, 120, 160, 120)
(10, 1)
(10, 1)
(10, 1)
(10, 1)


**Generate Training, Validation and Test datasets**

In [ ]:
## train select XX AD_F, XX AD_M, XX NC_F, and XX NC_M samples
num_train = np.round(0.8*num_samples/4).astype(int)
num_val = np.round(0.1*num_samples/4).astype(int)

AD_F_train = sample(AD_F_list, num_train)
AD_M_train = sample(AD_M_list, num_train)
NC_F_train = sample(NC_F_list, num_train)
NC_M_train = sample(NC_M_list, num_train)


train_list = AD_F_train + AD_M_train + NC_F_train + NC_M_train

Input_train = np.concatenate((AD_F_scans[AD_F_train],AD_M_scans[AD_M_train],NC_F_scans[NC_F_train],NC_M_scans[NC_M_train]), axis = 0)
Label_train = to_categorical(np.concatenate((AD_F_labels[AD_F_train],AD_M_labels[AD_M_train],NC_F_labels[NC_F_train],NC_M_labels[NC_M_train]), axis = 0))

# valid select XX AD_F, XX AD_M, XX NC_F, and XX NC_M samples
AD_F_val = sample(set(AD_F_list) - set(AD_F_train), num_val)
AD_M_val = sample(set(AD_M_list) - set(AD_M_train), num_val)
NC_F_val = sample(set(NC_F_list) - set(NC_F_train), num_val)
NC_M_val = sample(set(NC_M_list) - set(NC_M_train), num_val)

val_list = AD_F_val + AD_M_val + NC_F_val + NC_M_val

Input_val = np.concatenate((AD_F_scans[AD_F_val],AD_M_scans[AD_M_val],NC_F_scans[NC_F_val],NC_M_scans[NC_M_val]), axis = 0)
Label_val = to_categorical(np.concatenate((AD_F_labels[AD_F_val],AD_M_labels[AD_M_val],NC_F_labels[NC_F_val],NC_M_labels[NC_M_val]), axis = 0))

## test
AD_F_test = list(set(AD_F_list) - set(AD_F_train) - set(AD_F_val))
AD_M_test = list(set(AD_M_list) - set(AD_M_train) - set(AD_M_val))
NC_F_test = list(set(NC_F_list) - set(NC_F_train) - set(NC_F_val))
NC_M_test = list(set(NC_M_list) - set(NC_M_train) - set(NC_M_val))

test_list = list(set(total_list) - set(train_list) - set(val_list))

# test_list
Input_test = np.concatenate((AD_F_scans[AD_F_test],AD_M_scans[AD_M_test],NC_F_scans[NC_F_test],NC_M_scans[NC_M_test]), axis = 0)
Label_test = to_categorical(np.concatenate((AD_F_labels[AD_F_test],AD_M_labels[AD_M_test],NC_F_labels[NC_F_test],NC_M_labels[NC_M_test]), axis = 0))



In [ ]:
print(70*'-','\nTraining Dataset distribution\n',70*'-')
print('Number of \'AD_F\' samples:',len(AD_F_train))
print('Number of \'AD_M\' samples:',len(AD_M_train))
print('Number of \'NC_F\' samples:',len(NC_F_train))
print('Number of \'NC_M\' samples:',len(NC_M_train))
print('Total number of \'AD\' samples:',len(AD_F_train)+len(AD_M_train))
print('Total number of \'NC\' samples:',len(NC_F_train)+len(NC_M_train))
print('Total number of samples:',len(train_list))

print(70*'-','\nValidation Dataset distribution\n',70*'-')
print('Number of \'AD_F\' samples:',len(AD_F_val))
print('Number of \'AD_M\' samples:',len(AD_M_val))
print('Number of \'NC_F\' samples:',len(NC_F_val))
print('Number of \'NC_M\' samples:',len(NC_M_val))
print('Total number of \'AD\' samples:',len(AD_F_val)+len(AD_M_val))
print('Total number of \'NC\' samples:',len(NC_F_val)+len(NC_M_val))
print('Total number of samples:',len(val_list))

print(70*'-','\nTest Dataset distribution\n',70*'-')
print('Number of \'AD_F\' samples:',len(AD_F_list)-len(AD_F_train)-len(AD_F_val))
print('Number of \'AD_M\' samples:',len(AD_M_list)-len(AD_M_train)-len(AD_M_val))
print('Number of \'NC_F\' samples:',len(NC_F_list)-len(NC_F_train)-len(NC_F_val))
print('Number of \'NC_M\' samples:',len(NC_M_list)-len(NC_M_train)-len(NC_M_val))
print('Total number of \'AD\' samples:',len(AD_F_test)+len(AD_M_test))
print('Total number of \'NC\' samples:',len(NC_F_test)+len(NC_M_test))
print('Total number of samples:',len(test_list))

---------------------------------------------------------------------- 
Training Dataset distribution
 ----------------------------------------------------------------------
Number of 'AD_F' samples: 8
Number of 'AD_M' samples: 8
Number of 'NC_F' samples: 8
Number of 'NC_M' samples: 8
Total number of 'AD' samples: 16
Total number of 'NC' samples: 16
Total number of samples: 32
---------------------------------------------------------------------- 
Validation Dataset distribution
 ----------------------------------------------------------------------
Number of 'AD_F' samples: 1
Number of 'AD_M' samples: 1
Number of 'NC_F' samples: 1
Number of 'NC_M' samples: 1
Total number of 'AD' samples: 2
Total number of 'NC' samples: 2
Total number of samples: 4
---------------------------------------------------------------------- 
Test Dataset distribution
 ----------------------------------------------------------------------
Number of 'AD_F' samples: 1
Number of 'AD_M' samples: 1
Number of 'NC_F

**Define loss function**

In [ ]:
def loss_DSSIM(y_true, y_pred):
    ssim =tf.image.ssim(y_true, y_pred, 1.0)
    # tv = tf.image.total_variation(y_true - y_pred)
    mae = K.mean(tf.keras.losses.mean_absolute_error(y_true, y_pred))
    return K.mean((1.0 - ssim + mae))

def DSSIM(y_true, y_pred):
    ssim =tf.image.ssim(y_true, y_pred, 1.0)
    return K.mean(ssim)

def DPSNR(y_true, y_pred):
    psnr =tf.image.psnr(y_true, y_pred,1)
    return K.mean(psnr)

def dsc(y_true, y_pred):
    smooth_ = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth_) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth_)
    return score

# def dice_loss(y_true, y_pred):
    # loss = 1 - dsc(y_true, y_pred)
    # return loss

def dice_loss(y_true, y_pred):
  numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=-1)
  denominator = tf.reduce_sum(y_true + y_pred, axis=-1)

  return 1 - (numerator + 1) / (denominator + 1)

**Define performance measures**

In [ ]:
def pmeasure(y_true, y_pred):
  cm = confusion_matrix(y_true.ravel(), y_pred.ravel())
  tp_0 = cm[0][0]
  tn_0 = cm[1][1]
  fp_0 = cm[0][1]
  fn_0 = cm[1][0]

  sensitivity_0 = tp_0 / (tp_0 + fn_0)
  specificity_0 = tn_0 / (tn_0 + fp_0)

  f1_score_0 = 2 * tp_0 / (2 * (tp_0 + fp_0 + fn_0))
 

  return ({'Sensitivity': sensitivity_0,
           'Specificity': specificity_0,
           'F1-Score': f1_score_0})
  
def check_preds(ypred, ytrue):
    smooth = 1
    pred = np.ndarray.flatten(np.clip(ypred, 0, 1))
    gt = np.ndarray.flatten(np.clip(ytrue, 0, 1))
    intersection = np.sum(pred * gt) 
    union = np.sum(pred) + np.sum(gt)   
    return np.round((2 * intersection + smooth)/(union + smooth), decimals=5)

def auc(y_true, y_pred):
    smooth = 1
    y_pred_pos = np.round(np.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos
    y_pos = np.round(np.clip(y_true, 0, 1))
    y_neg = 1 - y_pos
    tp = np.sum(y_pos * y_pred_pos)
    tn = np.sum(y_neg * y_pred_neg)
    fp = np.sum(y_neg * y_pred_pos)
    fn = np.sum(y_pos * y_pred_neg)
    tpr = (tp + smooth) / (tp + fn + smooth)  # recall
    tnr = (tn + smooth) / (tn + fp + smooth)
    prec = (tp + smooth) / (tp + fp + smooth)  # precision
    return [tpr, tnr, prec]
  
def dice_score(y_true, y_pred, thres):
  dice = np.zeros(y_true.shape[0])
  dsc_thres = np.zeros_like(dice)
  recall = np.zeros_like(dice)
  precision = np.zeros_like(dice)

  for i in range(y_true.shape[0]):
    ds = dsc(y_true[i], y_pred[i])
    dice[i] = K.eval(ds)
    dsc_thres[i] = check_preds(y_pred[i] > thres, y_true[i])
    recall[i], _, precision[i] = auc(y_true[i], y_pred[i] > thres)
  
  dice = np.mean(dice)
  dsc_thres = np.mean(dsc_thres)
  recall = np.mean(recall)
  precision = np.mean(precision)

  return {'Dice score': dice, 'Dice score with threshold': dsc_thres, 'Recall': recall, 'Precision': precision}

def Show_Statistics(msg, Stat):
    print(msg.upper())
    print(70 * '-')
    print('Accuracy:', Stat[0])

    print('Sensitivity:', Stat[1])
    print('Specificity:', Stat[2])
    print('F1-Score:', Stat[3])

    print('Balance Accuracy:', Stat[4])
    print(70 * '-')



# Designing an Auto-Encoder-based classification model

In [ ]:
def BUS_Final_Model(num_filters=8,input_shape=(120, 160, 120, 1)):
    # Encoder Network
    encoder_input = Input(shape=input_shape, name='encoder_input')
    enc_l1 = Conv3D(num_filters, 15, activation='relu', name='encoder_layer1', padding='same')(encoder_input)
    enc_l1 = BatchNormalization()(enc_l1)
    enc_l1 = Conv3D(num_filters, 3, activation='relu', name='encoder_layer2', padding='same')(enc_l1)
    enc_l1 = BatchNormalization()(enc_l1)
    enc_l1 = Dropout(0.4)(enc_l1)
    enc_l1 = MaxPooling3D(pool_size=(8, 8, 8))(enc_l1)
    # enc_l1 = MaxPooling2D(pool_size=(4, 4))(enc_l1)

    # enc_l2 = Conv2D(2 * num_filters, 3, activation='relu', name='encoder_layer3', padding='same')(enc_l1)
    # enc_l2 = BatchNormalization()(enc_l2)
    # enc_l2 = Conv2D(2 * num_filters, 3, activation='relu', name='encoder_layer4', padding='same')(enc_l2)
    # enc_l2 = BatchNormalization()(enc_l2)
    # # enc_l2 = Dropout(0.4)(enc_l2)
    # enc_l2 = MaxPooling2D(pool_size=(2, 2))(enc_l2)

    # enc_l3 = Conv2D(4 * num_filters, 3, activation='relu', name='encoder_layer5', padding='same')(enc_l2)
    # enc_l3 = BatchNormalization()(enc_l3)
    # enc_l3 = Conv2D(4 * num_filters, 3, activation='relu', name='encoder_layer6', padding='same')(enc_l3)
    # enc_l3 = BatchNormalization()(enc_l3)
    # # enc_l3 = Dropout(0.4)(enc_l3)
    # enc_l3 = MaxPooling2D(pool_size=(2, 2))(enc_l3)

    # enc_l4 = Conv2D(8 * num_filters, 3, activation='relu', name='encoder_layer7', padding='same')(enc_l3)
    # enc_l4 = BatchNormalization()(enc_l4)
    # enc_l4 = Conv2D(8 * num_filters, 3, activation='relu', name='encoder_layer8', padding='same')(enc_l4)
    # enc_l4 = BatchNormalization()(enc_l4)
    # # enc_l4 = Dropout(0.4)(enc_l4)
    # enc_l4 = MaxPooling2D(pool_size=(2, 2))(enc_l4)

    # enc_l5 = Conv2D(16 * num_filters, 3, activation='relu', name='encoder_layer9', padding='same')(enc_l4)
    # enc_l5 = BatchNormalization()(enc_l5)
    # enc_l5 = Conv2D(16 * num_filters, 3, activation='relu', name='encoder_layer10', padding='same')(enc_l5)
    # enc_l5 = BatchNormalization()(enc_l5)
    # enc_l5 = Dropout(0.4)(enc_l5)
    # enc_l5 = MaxPooling2D(pool_size=(2, 2))(enc_l5)

    # encoder_output = Conv2D(32 * num_filters, 3, activation='relu', name='encoder_output', padding='same')(enc_l5)
    encoder_output = Conv3D(num_filters, 3, activation='relu', name='encoder_output', padding='same')(enc_l1)

    # Classifier Network
    flat = Flatten()(encoder_output)
    class_l1 = Dense(8, activation='relu', name='class_layer1')(flat)
    # class_l1 = BatchNormalization()(class_l1)
    # drop_c1 = Dropout(0.4)(class_l1)
    # class_l2 = Dense(512, activation='relu', name='class_layer2')(drop_c1)
    # class_l2 = BatchNormalization()(class_l2)
    # drop_c2 = Dropout(0.4)(class_l2)
    # class_l3 = Dense(256, activation='relu', name='class_layer3')(drop_c2)
    # class_l3 = BatchNormalization()(class_l3)
    # drop_c3 = Dropout(0.4)(class_l3)
    # class_output = Dense(4, activation='softmax', name='class_output')(drop_c3)
    class_output = Dense(2, activation='softmax', name='class_output')(class_l1)

    # # Decoder Network
    # # dec_l1 = UpSampling2D(size=(2, 2))(concatenate([enc_l5, encoder_output], axis=3))
    # dec_l1 = UpSampling2D(size=(2, 2))(encoder_output)
    # dec_l1 = Conv2D(16 * num_filters, 3, activation='relu', name='decoder_layer1', padding='same')(dec_l1)
    # dec_l1 = BatchNormalization()(dec_l1)
    # dec_l1 = Conv2D(16 * num_filters, 3, activation='relu', name='decoder_layer2', padding='same')(dec_l1)
    # dec_l1 = BatchNormalization()(dec_l1)
    # # dec_l1 = Dropout(0.4)(dec_l1)

    # # dec_l2 = UpSampling2D(size=(2, 2))(concatenate([enc_l4, dec_l1], axis=3))
    # dec_l2 = UpSampling2D(size=(2, 2))(dec_l1)
    # dec_l2 = Conv2D(8 * num_filters, 3, activation='relu', name='decoder_layer3', padding='same')(dec_l2)
    # # dec_l2 = BatchNormalization()(dec_l2)
    # dec_l2 = Conv2D(8 * num_filters, 3, activation='relu', name='decoder_layer4', padding='same')(dec_l2)
    # dec_l2 = BatchNormalization()(dec_l2)
    # # dec_l2 = Dropout(0.4)(dec_l2)

    # # dec_l3 = UpSampling2D(size=(2, 2))(concatenate([enc_l3, dec_l2], axis=3))
    # dec_l3 = UpSampling2D(size=(2, 2))(dec_l2)
    # dec_l3 = Conv2D(2 * num_filters, 3, activation='relu', name='decoder_layer5', padding='same')(dec_l3)
    # # dec_l3 = BatchNormalization()(dec_l3)
    # dec_l3 = Conv2D(2 * num_filters, 3, activation='relu', name='decoder_layer6', padding='same')(dec_l3)
    # dec_l3 = BatchNormalization()(dec_l3)
    # # dec_l3 = Dropout(0.4)(dec_l3)

    # # dec_l4 = UpSampling2D(size=(2, 2))(concatenate([enc_l2, dec_l3], axis=3))
    # dec_l4 = UpSampling2D(size=(2, 2))(dec_l3)
    # dec_l4 = Conv2D(num_filters, 3, activation='relu', name='decoder_layer7', padding='same')(dec_l4)
    # # dec_l4 = BatchNormalization()(dec_l4)
    # dec_l4 = Conv2D(num_filters, 3, activation='relu', name='decoder_layer8', padding='same')(dec_l4)
    # dec_l4 = BatchNormalization()(dec_l4)
    # # dec_l4 = Dropout(0.4)(dec_l4)

    # # dec_l5 = UpSampling2D(size=(2, 2))(concatenate([enc_l1, dec_l4], axis=3))
    # dec_l5 = UpSampling2D(size=(2, 2))(dec_l4)
    # dec_l5 = Conv2D(num_filters, 3, activation='relu', name='decoder_layer10', padding='same')(dec_l5)
    # # dec_l5 = BatchNormalization()(dec_l5)
    # dec_l5 = Conv2D(num_filters, 3, activation='relu', name='decoder_layer9', padding='same')(dec_l5)
    # dec_l5 = BatchNormalization()(dec_l5)
    # dec_l5 = Dropout(0.4)(dec_l5)

    # decoder_output = Conv2D(input_shape[2], 1, activation='relu', name='decoder_output', padding='same')(dec_l5)

    # model = Model(inputs=[encoder_input], outputs=[class_output, decoder_output])  # class_output, decoder_output,
    model = Model(inputs=[encoder_input], outputs=[class_output])  # class_output, decoder_output,

    # Compiling model
    # model.compile(optimizer=tf.keras.optimizers.Adam(),#learning_rate=0.0001, beta_1=0.9, beta_2=0.999,
    #                                                   #epsilon=1e-07, amsgrad=False),
    #               loss= {'class_output': 'categorical_crossentropy', 'decoder_output': 'mae'}, #loss_DSSIM},
    #               loss_weights={'class_output': 0.01,  'decoder_output': 0.99},
    #               metrics={'class_output': 'categorical_accuracy',  'decoder_output': 'mae'})#[DSSIM, DPSNR, 'mae']})
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss= {'class_output': 'categorical_crossentropy'}, #loss_DSSIM},
                  metrics={'class_output': 'categorical_accuracy'})#[DSSIM, DPSNR, 'mae']})    
    model.summary()                        
    return model

In [ ]:
model = BUS_Final_Model()
# model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, 120, 160, 120, 1) 0         
_________________________________________________________________
encoder_layer1 (Conv3D)      (None, 120, 160, 120, 8)  27008     
_________________________________________________________________
batch_normalization (BatchNo (None, 120, 160, 120, 8)  32        
_________________________________________________________________
encoder_layer2 (Conv3D)      (None, 120, 160, 120, 8)  1736      
_________________________________________________________________
batch_normalization_1 (Batch (None, 120, 160, 120, 8)  32        
_________________________________________________________________
dropout (Dropout)            (None, 120, 160, 120, 8)  0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 15, 20, 15, 8)     0     

In [ ]:
Input_train = np.squeeze(Input_train)
Input_train = np.expand_dims(Input_train, axis=4)
print(Input_train.shape)

Input_val = np.expand_dims(Input_val, axis=4)
print(Input_val.shape)

Input_test = np.expand_dims(Input_test, axis=4)
print(Input_test.shape)

# out=model.predict(Input_train)
# print(out.shape)

(32, 120, 160, 120, 1)
(4, 120, 160, 120, 1)
(4, 120, 160, 120, 1)


ResourceExhaustedError: ignored

# Model Training

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=10)
checkpoint = ModelCheckpoint('models\\model-best.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')

# es = EarlyStopping(monitor='val_class_output_categorical_accuracy', mode='max', verbose=0, patience=10)

# checkpoint = ModelCheckpoint('models\\model-best.h5', verbose=1,
#                              monitor='val_class_output_categorical_accuracy',
#                              save_best_only=True, mode='auto')
  
# history = model.fit({'encoder_input': Input_train}, {'class_output': Label_train,'decoder_output': Input_train},             
#                     validation_data=({'encoder_input': Input_val}, {'class_output': Label_val, 'decoder_output': Input_val}),                                      
#                     batch_size=10, epochs=100, shuffle=True, verbose=2, callbacks=[checkpoint, es])

history = model.fit({'encoder_input': Input_train}, {'class_output': Label_train},
                    validation_data=({'encoder_input': Input_val}, {'class_output': Label_val}),
                    batch_size=1, epochs=100, shuffle=True, verbose=2, callbacks=[checkpoint, es])


Epoch 1/100
32/32 - 86s - loss: 7.4903 - categorical_accuracy: 0.7188 - val_loss: 0.7869 - val_categorical_accuracy: 0.5000

Epoch 00001: val_loss improved from inf to 0.78693, saving model to models\model-best.h5
Epoch 2/100
32/32 - 58s - loss: 0.8845 - categorical_accuracy: 0.5000 - val_loss: 0.6933 - val_categorical_accuracy: 0.5000

Epoch 00002: val_loss improved from 0.78693 to 0.69326, saving model to models\model-best.h5
Epoch 3/100
32/32 - 58s - loss: 0.6934 - categorical_accuracy: 0.5000 - val_loss: 0.6933 - val_categorical_accuracy: 0.5000

Epoch 00003: val_loss improved from 0.69326 to 0.69325, saving model to models\model-best.h5
Epoch 4/100
32/32 - 58s - loss: 0.6935 - categorical_accuracy: 0.5000 - val_loss: 0.6932 - val_categorical_accuracy: 0.5000

Epoch 00004: val_loss improved from 0.69325 to 0.69324, saving model to models\model-best.h5
Epoch 5/100
32/32 - 57s - loss: 0.6935 - categorical_accuracy: 0.5000 - val_loss: 0.6933 - val_categorical_accuracy: 0.5000

Epoch 0

# Performance Evaluation

In [ ]:
del model
model = load_model('models\model-best.h5')              
# [Label_train_pred, decoder_output_train_pred] = model.predict(Input_train, batch_size=1, verbose=0)
# [Label_val_pred, decoder_output_val_pred] = model.predict(Input_val, batch_size=1, verbose=0)
# [Label_test_pred, decoder_output_test_pred] = model.predict(Input_test, batch_size=1, verbose=0)
Label_train_pred = model.predict(Input_train, batch_size=1, verbose=0)
Label_val_pred = model.predict(Input_val, batch_size=1, verbose=0)
Label_test_pred = model.predict(Input_test, batch_size=1, verbose=0)

In [ ]:
Training_Stats = []
# train Measures
train_acc = accuracy_score(Label_train.argmax(axis=1), Label_train_pred.argmax(axis=1))
train_spe = pmeasure(Label_train.argmax(axis=1), Label_train_pred.argmax(axis=1))['Sensitivity']
train_sen = pmeasure(Label_train.argmax(axis=1), Label_train_pred.argmax(axis=1))['Specificity']
train_f1 = pmeasure(Label_train.argmax(axis=1), Label_train_pred.argmax(axis=1))['F1-Score']

train_bacc = balanced_accuracy_score(Label_train.argmax(axis=1), Label_train_pred.argmax(axis=1))

Training_Stats.append([train_acc,
                       train_sen, 
                       train_spe, 
                       train_f1, 
                       train_bacc])


In [ ]:
Validation_Stats = []
# val Measures
val_acc = accuracy_score(Label_val.argmax(axis=1), Label_val_pred.argmax(axis=1))
val_spe = pmeasure(Label_val.argmax(axis=1), Label_val_pred.argmax(axis=1))['Sensitivity']
val_sen = pmeasure(Label_val.argmax(axis=1), Label_val_pred.argmax(axis=1))['Specificity']
val_f1 = pmeasure(Label_val.argmax(axis=1), Label_val_pred.argmax(axis=1))['F1-Score']

val_bacc = balanced_accuracy_score(Label_val.argmax(axis=1), Label_val_pred.argmax(axis=1))

Validation_Stats.append([val_acc,
                       val_sen, 
                       val_spe, 
                       val_f1, 
                       val_bacc])


In [ ]:
Testing_Stats = []
# Test Measures
tst_acc = accuracy_score(Label_test.argmax(axis=1), Label_test_pred.argmax(axis=1))
tst_spe = pmeasure(Label_test.argmax(axis=1), Label_test_pred.argmax(axis=1))['Sensitivity']
tst_sen = pmeasure(Label_test.argmax(axis=1), Label_test_pred.argmax(axis=1))['Specificity']
tst_f1 = pmeasure(Label_test.argmax(axis=1), Label_test_pred.argmax(axis=1))['F1-Score']

tst_bacc = balanced_accuracy_score(Label_test.argmax(axis=1), Label_test_pred.argmax(axis=1))

Testing_Stats.append([tst_acc,
                       tst_sen, 
                       tst_spe, 
                       tst_f1, 
                       tst_bacc])


In [ ]:
Train_Statistics = np.asarray(Training_Stats)
Val_Statistics = np.asarray(Validation_Stats)
Test_Statistics = np.asarray(Testing_Stats)

# Show Classification/Reconstruction Statistics
Show_Statistics('Training Results', Train_Statistics[0])
Show_Statistics('Validation Results', Val_Statistics[0])
Show_Statistics('Test Results', Test_Statistics[0])